In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.svm import SVC
import numpy as np
from sklearn import preprocessing
import xgboost
from catboost import CatBoostClassifier
from xgboost.sklearn import XGBClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.utils import shuffle
from utilities import Encode_Standarize, get_average_matrix, pretty_print_confusion_matrix, do_cross_validation, calculate_classification_report, draw_confusion_matrix

import os
import numpy as np
import warnings
warnings.filterwarnings("ignore")
!pip install numpy==1.23.1
!pip install ipython-autotime
%load_ext autotime
#  %unload_ext autotime     # to turn it off

## Experiments 

### Cross Validation

In [ ]:

df_clean = pd.read_csv("./dataset/SampleData746.csv")
print(df_clean.shape)
#display (df_clean.head(-5))


X = df_clean.drop(["SampleName","label"], axis=1)
y = df_clean["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.16667, random_state=42)
#train, test = train_test_split(df3,  test_size=0.17,  random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

print("=====================================")
print(X_train.shape[0] + X_test.shape[0])
print(y_train.value_counts()/12424.0)
print("============================================================")
print(y_test.value_counts()/2485.0)

In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier


seed = 42
Classifer_Name = [
    "CatBoost", 
    "xGBoost",
    "RandomForest",
    "DecisionTree",
    "GaussianNB",
    "LinearDiscriminantAnalysis",
    "MLPClassifier",
]

Classifier = [
    CatBoostClassifier(verbose = False, task_type="GPU", devices='0:1', random_seed=42),
    XGBClassifier(tree_method='gpu_hist', seed=42),
    RandomForestClassifier(max_depth=5, random_state=42),
    DecisionTreeClassifier(random_state=42),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    MLPClassifier(random_state=42, max_iter=300),
    
]
 
# read sample master datset includes train and test
df4 = pd.read_csv("./dataset2/SampleData746.csv") 
print(df4.shape)
display(df4.head(-5))

df5 = Encode_Standarize(df4)

df5 = shuffle(df5)

X = df5.drop(["SampleName","label"], axis=1)
y = df5["label"]


X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.16667, random_state=42)

# display(X_train.head(-5))
# display(y_train.head(-5))
# display(X_train_scaled.head(-5))
# display(X_test.head(-5))
# display(X_test_scaled.head(-5))

# Loop through all classifers and accumulate performance results
cv_nfolds = 3
clfs_results = {}
for clfname, clf in zip(Classifer_Name,Classifier):
    results = do_cross_validation(clf, X_train, y_train,  cv_nfolds)
    results_df = pd.DataFrame(results["result"])
    clfs_results[clfname] = results
 

### RESULTS - Classifers Cross Validation Results

In [ ]:
np.set_printoptions(precision=3)

print()
print ("CatBoost Cross validation fold results:")
CB_df_results = pd.DataFrame(clfs_results["CatBoost"]["result"])
display(CB_df_results)

print("average performace over all Foldes")
display(CB_df_results.iloc[:,0:4].mean())
CB_avg_conf_mat = clfs_results["CatBoost"]["average_matrix"]
print()
print("Average confusion matrix for CatBoost")
display(CB_avg_conf_mat)

CB_performance_df = calculate_classification_report(CB_avg_conf_mat, 5)
print()
print("Performance per class")
display(CB_performance_df) 


print()
print ("xGBoost Cross validation fold results:")
xGB_df_results = pd.DataFrame(clfs_results["xGBoost"]["result"])
display(xGB_df_results)

print("average performace over all Foldes")
display(xGB_df_results.iloc[:,0:4].mean())
xGB_avg_conf_mat = clfs_results["xGBoost"]["average_matrix"]
print()
print("Average confusion matrix for xGBoost")
display(xGB_avg_conf_mat)

xGB_performance_df = calculate_classification_report(xGB_avg_conf_mat, 5)
print()
print("Performance per class")
display(xGB_performance_df) 
#print(class_idx_to_name) # Show class mappings

print()
print ("RandomForest Cross validation fold results:")
RF_df_results = pd.DataFrame(clfs_results["RandomForest"]["result"])
display(RF_df_results)

print("average performace over all Foldes")
display(RF_df_results.iloc[:,0:4].mean())
RF_avg_conf_mat = clfs_results["RandomForest"]["average_matrix"]
print()
print("Average confusion matrix for RandomForest")
display(RF_avg_conf_mat)

RF_performance_df = calculate_classification_report(RF_avg_conf_mat, 5)
print()
print("Performance per class")
display(RF_performance_df)

print()
print ("DecisionTree Cross validation fold results:")
DT_df_results = pd.DataFrame(clfs_results["DecisionTree"]["result"])
display(RF_df_results)

print("average performace over all Foldes")
display(DT_df_results.iloc[:,0:4].mean())
DT_avg_conf_mat = clfs_results["DecisionTree"]["average_matrix"]
print()
print("Average confusion matrix for DecisionTreet")
display(DT_avg_conf_mat)

DT_performance_df = calculate_classification_report(DT_avg_conf_mat, 5)
print()
print("Performance per class")
display(DT_performance_df)

# ###################################################################
# GaussianNB
# ###################################################################
print()
print ("GaussianNB Cross validation fold results:")
GNB_df_results = pd.DataFrame(clfs_results["GaussianNB"]["result"])
display(GNB_df_results)

print("average performace over all Foldes")
display(GNB_df_results.iloc[:,0:4].mean())
GNB_avg_conf_mat = clfs_results["GaussianNB"]["average_matrix"]
print()
print("Average confusion matrix for GaussianNB")
display(GNB_avg_conf_mat)

GNB_performance_df = calculate_classification_report(GNB_avg_conf_mat, 5)
print()
print("Performance per class")
display(GNB_performance_df)
GNB_performance_df.to_csv('GNB_performance_cv.csv', encoding='utf-8', index=False)
# ###################################################################
# LinearDiscriminantAnalysis
# ###################################################################
print()
print ("LinearDiscriminantAnalysis Cross validation fold results:")
LDA_df_results = pd.DataFrame(clfs_results["LinearDiscriminantAnalysis"]["result"])
display(LDA_df_results)

print("average performace over all Foldes")
display(LDA_df_results.iloc[:,0:4].mean())
LDA_avg_conf_mat = clfs_results["LinearDiscriminantAnalysis"]["average_matrix"]
print()
print("Average confusion matrix for LinearDiscriminantAnalysis")
display(LDA_avg_conf_mat)

LDA_performance_df = calculate_classification_report(LDA_avg_conf_mat, 5)
print()
print("Performance per class")
display(LDA_performance_df)


# ###################################################################
# MLPClassifier
# ###################################################################
print()
print ("MLPClassifier Cross validation fold results:")
MLP_df_results = pd.DataFrame(clfs_results["MLPClassifier"]["result"])
display(MLP_df_results)

print("average performace over all Foldes")
display(MLP_df_results.iloc[:,0:4].mean())
MLP_avg_conf_mat = clfs_results["MLPClassifier"]["average_matrix"]
print()
print("Average confusion matrix for MLPClassifier")
display(MLP_avg_conf_mat)

MLP_performance_df = calculate_classification_report(MLP_avg_conf_mat, 5)
print()
print("Performance per class")
display(MLP_performance_df)
MLP_performance_df.to_csv('MLP_performance_cv.csv', encoding='utf-8', index=False)



### Test with Independent Dataset

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
labels = ['Cytoplasm', 'Endoplasmic_Reticulum', 'Extracellular', 'Mitochondria', 'Nucleus']

print("==================== CatBoost ==============================================")
CBclf = CatBoostClassifier(iterations=1000,depth=4,learning_rate=0.1, task_type="GPU", devices='0:1', loss_function='MultiClass',verbose=False,random_seed=42)
CBclf.fit(X_train, y_train) # train on full training dataset (5/6)
print(CBclf.classes_)

# test with unseen data (1/6)
y_pred = CBclf.predict(X_test)
print(confusion_matrix(y_test, y_pred))

acc = accuracy_score(y_test, y_pred)

print(f'Accuracy: {acc:.2f}')
cr = classification_report(y_test, y_pred, target_names=labels)
print(cr)

CatBoost_cmat = confusion_matrix(y_test, y_pred)
print(CatBoost_cmat)
display(pd.DataFrame(calculate_classification_report(CatBoost_cmat, 5)))

print()
print("===================== XGBBoost ===============================================")
xgb = XGBClassifier(tree_method='gpu_hist')
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
print(confusion_matrix(y_test, xgb_pred))
print(round(accuracy_score(y_test, xgb_pred),2)*100)

cr = classification_report(y_test, xgb_pred, target_names=labels)
print(cr)

XGBBoost_cmat = confusion_matrix(y_test, xgb_pred)
print(XGBBoost_cmat)
display(pd.DataFrame(calculate_classification_report(XGBBoost_cmat, 5)))


print()
print("ْ===================== Decision Tree ===========================================")
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)
print(confusion_matrix(y_test, dt_pred))
print(round(accuracy_score(y_test, dt_pred),2)*100)

cr = classification_report(y_test, dt_pred, target_names=labels)
print(cr)

dt_cmat = confusion_matrix(y_test, dt_pred)
print(dt_cmat)
display(pd.DataFrame(calculate_classification_report(dt_cmat, 5)))


    # GaussianNB(),
    # LinearDiscriminantAnalysis(),
    # MLPClassifier(random_state=42, max_iter=300),


print()
print("===================== GaussianNB ===========================================")
GNB = GaussianNB()
GNB.fit(X_train, y_train)
GNB_pred = GNB.predict(X_test)
print(confusion_matrix(y_test, GNB_pred))
print(round(accuracy_score(y_test, GNB_pred),2)*100)

cr = classification_report(y_test, GNB_pred, target_names=labels)
print(cr)

GNB_cmat = confusion_matrix(y_test, GNB_pred)
print(GNB_cmat)
display(pd.DataFrame(calculate_classification_report(GNB_cmat, 5)))
GNB_performance_ind = pd.DataFrame(calculate_classification_report(GNB_cmat, 5))
GNB_performance_ind.to_csv('GNB_performance_ind.csv', encoding='utf-8', index=False)


print()
print("===================== MLPClassifier ===========================================")
MLP = MLPClassifier(random_state=42, max_iter=300)
MLP.fit(X_train, y_train)
MLP_pred = MLP.predict(X_test)
print(confusion_matrix(y_test, MLP_pred))
print(round(accuracy_score(y_test, MLP_pred),2)*100)

cr = classification_report(y_test, MLP_pred, target_names=labels)
print(cr)

MLP_cmat = confusion_matrix(y_test, MLP_pred)
print(MLP_cmat)
display(pd.DataFrame(calculate_classification_report(MLP_cmat, 5)))
MLP_performance_ind = pd.DataFrame(calculate_classification_report(MLP_cmat, 5))
MLP_performance_ind.to_csv('MLP_performance_ind.csv', encoding='utf-8', index=False)



# Stacking Classifers

In [ ]:
df4 = pd.read_csv("./dataset2/SampleData746.csv") # read master datset includes train and test
print(df4.shape)
display(df4.head(-5))

df5 = Encode_Standarize(df4)

df5 = shuffle(df5)

X = df5.drop(["SampleName","label"], axis=1)
y = df5["label"]


X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.16667, random_state=42)

In [ ]:
#from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

estimators = [('CB',  CatBoostClassifier(iterations=1000,depth=4,learning_rate=0.1, task_type="GPU", devices='0:1', loss_function='MultiClass',verbose=False,random_seed=42)),
              ('XGB', XGBClassifier(tree_method='gpu_hist')),
              ('GNB',GaussianNB()),
              ('LDA', LinearDiscriminantAnalysis()),
              ('MLP',MLPClassifier(random_state=42, max_iter=300)),
             ]
clf = StackingClassifier( estimators=estimators, 
                          #final_estimator=CatBoostClassifier(learning_rate=0.1, loss_function='MultiClass',verbose=False,random_seed=42))
                          final_estimator=XGBClassifier())

clf.fit(X_train, y_train)

y_pred_train = clf.predict(X_train)
y_prob_train = clf.predict_proba(X_train)[:,1]

y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:,1]

print('Classification report for test:\n',classification_report(y_test,y_pred))


In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)